In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np  
from earlystopping import EarlyStopping
from models import *
from utils import *

Using backend: pytorch


In [2]:
#cerevisiae,silico,coli
name='coli'
expression,network,tfs=load_data(name)

In [4]:
scaler = MinMaxScaler()
data=scaler.fit_transform(expression).T

In [8]:
df_expression=pd.DataFrame(data.T, columns=expression.columns)

In [9]:
src_gene=network[0].values
dst_gene=network[1].values

In [10]:
src_expression_df=df_expression[src_gene]
dst_expression_df=df_expression[dst_gene]

In [11]:
input_x=torch.tensor(src_expression_df.T.values,dtype=torch.float32)
input_y=torch.tensor(dst_expression_df.T.values,dtype=torch.float32)

In [15]:
inputs=torch.cat([input_x,input_y],1)

In [21]:
pos_number=network[network[4]==1].shape[0]
negtive_number=network[network[4]==0].shape[0]

In [22]:
test_ratio=0.1
val_ratio=0.1

In [23]:
pos_maks_result= pos_masks(pos_number,negtive_number, test_ratio)

In [24]:
train_pos_mask=pos_maks_result['pos_masks'][0]
test_pos_mask=pos_maks_result['pos_masks'][1]
val_pos_mask=pos_maks_result['pos_masks'][2]

In [25]:
test_pos_id=pos_maks_result['test_id']

In [26]:
test_neg_mask,val_neg_mask,train_neg_set=test_val_neg_maks(pos_number,negtive_number,test_ratio,val_ratio)

In [34]:
train_size = pos_maks_result['train_size']

In [35]:
lr=0.001
weight_decay=0.
in_feats=input_x.shape[1]
n_hidden=128
out_dim=1

In [36]:
model=fusion1(in_feats,n_hidden,out_dim)

In [37]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr,weight_decay=weight_decay)

In [39]:
patience = 20
early_stopping = EarlyStopping(patience,path='conv.pt', verbose=True) 

In [40]:
for epoch in range(1000000):
    model.train()
    
    train_neg_mask=train_neg_mask_build(pos_number,negtive_number,train_neg_set,train_size)
    
    logits = model(input_x,input_y,inputs)
#     loss,train_acc = compute_loss(logits[train_pos_mask],logits[train_neg_mask])
    loss= binary_loss(logits[train_pos_mask].view(-1),logits[train_neg_mask].view(-1))
    
    val_loss,acc = binary_val(model,input_x,input_y,inputs,val_pos_mask,val_neg_mask)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
#     scheduler.step()
    if epoch % 5 == 0:
        print('In epoch {}, train_loss: {},val_loss:{},val_acc:{}'.format(epoch, loss,val_loss,acc))
#         print('In epoch {}, train_loss: {}, train_acc:{},val_loss:{},val_acc:{}'.format(epoch, loss,train_acc,val_loss,acc))
    early_stopping(val_loss, model)
    if early_stopping.early_stop:
        print  ("Early stopping")
        break 
with torch.no_grad():

    pred = model(input_x,input_y,inputs)
    test_loss=binary_loss(pred[test_pos_mask].view(-1),pred[test_neg_mask].view(-1))
    test_acc=compute_auc(pred[test_pos_mask].view(-1),pred[test_neg_mask].view(-1))
    print(test_loss)
    print(test_acc)

In epoch 0, train_loss: 0.6914793848991394,val_loss:0.6926318407058716,val_acc:0.5271715323765106
In epoch 5, train_loss: 0.6342646479606628,val_loss:0.6226556897163391,val_acc:0.7292509469452962
In epoch 10, train_loss: 0.6004579663276672,val_loss:0.5817379355430603,val_acc:0.7730874281738773
In epoch 15, train_loss: 0.5710968375205994,val_loss:0.5449512004852295,val_acc:0.8154422943131747
In epoch 20, train_loss: 0.5428081750869751,val_loss:0.515661358833313,val_acc:0.8335888582545286
In epoch 25, train_loss: 0.5183410048484802,val_loss:0.5003213882446289,val_acc:0.8409067484346414
In epoch 30, train_loss: 0.49506819248199463,val_loss:0.48694103956222534,val_acc:0.8492939782009328
In epoch 35, train_loss: 0.4684229791164398,val_loss:0.4806852638721466,val_acc:0.8501765054497668
In epoch 40, train_loss: 0.4379933178424835,val_loss:0.4739850163459778,val_acc:0.8542734932618723
In epoch 45, train_loss: 0.4236830174922943,val_loss:0.46593567728996277,val_acc:0.8590404287665232
In epoch 5